<a href="https://colab.research.google.com/github/ViniciusCastillo/BootcampAlura_ProjetoModulo5/blob/main/Notebooks/Bases_Funcoes_Classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook dedicado ao tratamento de dados, criação de funções e classes

Neste notebook iremos importar as bibliotecas necessárias, fazer o tratamento inicial dos dados e criar as funções e classes necessárias para as avaliações do projeto

## Importando as bibliotecas

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, classification_report, ConfusionMatrixDisplay, f1_score
from sklearn.base import BaseEstimator, TransformerMixin
from joblib import dump, load

## Importando e tratando os dados


### Importando a base de dados
A base original está neste desafio do [Kaggle](https://www.kaggle.com/S%C3%ADrio-Libanes/covid19)

In [ ]:
dados = pd.read_excel("https://github.com/ViniciusCastillo/BootcampAlura_ProjetoModulo5/blob/main/dados/Kaggle_Sirio_Libanes_ICU_Prediction.xlsx?raw=true")

### Tratando os dados

#### Primeiro passo: criar as bases dados_tratados e dados_LE
Aqui será realizado alguns tratamentos:
* Retirar as linhas com marcação de UTI = 1
* Remarcar a coluna ICU (UTI em inglês), conforme qual visita de pacientes (PATIENT_VISIT_IDENTIFIER) chegou algum momento na UTI, independente da janela (campo WINDOW), em todas as janelas.
* Tratando os dados que não estão disponíveis com base na medição anterior ou posterior.
** Caso ainda sobre dados indisponível, as linhas onde eles estão serão excluídas. 
** Isso se o limite de 10% da base for atendido, caso contrário irá informar o problema e não excluirá os dados.
* Reinicia o index para a numeração ficar de 0 até o número de elementos.

A base dados_LE irá alterar o formato do campo AGE_PERCENTIL para valores numéricos com base na função [LabelEncoder() do scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).

Caso a base passe a ter mais de um campo categórico de texto a base dados_LE não será criada e será comunicado o ocorrido.

In [ ]:
print("Tratamento da Base com Label Encoder")

pacientes_UTI = dados[['PATIENT_VISIT_IDENTIFIER','ICU']].query('ICU == 1').groupby('PATIENT_VISIT_IDENTIFIER').min()
dados_tratados = dados.query('ICU != 1').drop('ICU', axis=1)
dados_tratados = dados_tratados.join(pacientes_UTI, on='PATIENT_VISIT_IDENTIFIER', how='left')
dados_tratados['ICU'] = dados_tratados['ICU'].fillna(0) 
print("\nRemovemos as linhas com ICU(UTI) igual a 1 e remarcamos a coluna com base no PATIENT_VISIT_IDENTIFIER que chegou na UTI")
print(f"Distribuição de ICU na base tratada (%)\n{dados_tratados['ICU'].value_counts(normalize=True)*100}")

features_continuas_colunas = dados_tratados.iloc[:, 13:-2].columns
features_continuas = dados_tratados.groupby("PATIENT_VISIT_IDENTIFIER",as_index=False)[features_continuas_colunas].fillna(method='bfill').fillna(method='ffill')
features_categoricas = dados_tratados.iloc[:, :13]
saida = dados_tratados.iloc[:, -2:]
dados_tratados = pd.concat([features_categoricas, features_continuas, saida], ignore_index=True,axis=1)
dados_tratados.columns = dados.columns
print("\nAjustamos os valores continuos que estavam com Nam para o valor anterior ou posterior")

descricao = dados_tratados.describe().T
colunas_sem_variacao = descricao[descricao['min'] == descricao['max']].index
if len(colunas_sem_variacao) !=0:
  dados_tratados.drop(colunas_sem_variacao, axis=1)
  print("\nRemovemos as colunas que os valores são iguais para todas as linhas")

linhas_com_nam = dados_tratados.describe(include='all').loc['count'].max()-dados_tratados.describe(include='all').loc['count'].min()
if linhas_com_nam !=0:
  if linhas_com_nam <= len(dados_tratados)*.1:
    dados_tratados.dropna(inplace=True)
    print(f"\nAs linhas ainda com Nam ({linhas_com_nam} linhas, {linhas_com_nam/len(dados_tratados):.2%} do total) foram eliminadas")
  else:
    print(f"\nTemos linhas ainda com Nam ({linhas_com_nam} linhas, {linhas_com_nam/len(dados_tratados):.2%} do total) precisam ser tratadas")

dados_tratados.reset_index(drop=True, inplace=True)
print(f"\nO index foi resetado: {dados_tratados.index}")

colunas_categoricas = list(set(dados_tratados.columns)-set(dados_tratados.describe().columns)-{'WINDOW'})
if len(colunas_categoricas) ==1:
  LE = LabelEncoder()
  LE.fit(np.ravel(dados_tratados[colunas_categoricas]))
  dados_LE = dados_tratados.copy()
  dados_LE[colunas_categoricas] = LE.transform(np.ravel(dados_LE[colunas_categoricas]))
  print(f"\nColuna com objeto categórico ({colunas_categoricas[0]}) foi transformada em numérica no DataFrame dados_LE.\nFormato: {dados_LE.shape}")
else:
  print(f"\nColunas com objetos categóricos precisam ser tratados: {', '.join(colunas_categoricas)}")
  
print(f"\nFormato final do DataFrame dados_tratados: {dados_tratados.shape}")

#### Segundo passo: criar a base dados_OHE
A base dados_OHE irá alterar o formato do campo AGE_PERCENTIL para valores numéricos com base na função [OneHotEncoder() do scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html). Este modelo transforma o campo em diversas colunas, uma para cada categoria.

Caso a base passe a ter mais de um campo categórico de texto a base dados_OHE não será criada e será comunicado o ocorrido.

In [ ]:
print(f"Tratamento da Base com One Hot Encoder")

if len(colunas_categoricas) ==1:
  categorica = np.array(dados_tratados[colunas_categoricas]).reshape(-1, 1)
  OHE = OneHotEncoder()
  categorica_OHE = pd.DataFrame(OHE.fit_transform(categorica).toarray())
  dados_OHE = pd.concat([dados_tratados.drop(colunas_categoricas, axis=1), categorica_OHE], ignore_index=True, axis=1)
  colunas = list(dados_tratados.columns)
  colunas.remove(colunas_categoricas[0])
  colunas_novas = list(dados_tratados[colunas_categoricas[0]].unique())
  colunas_novas.sort()
  colunas.extend(colunas_novas)
  dados_OHE.columns = colunas
  print(f"\nTrocamos o campo AGE_PERCENTIL pelos campos binários {', '.join(colunas_novas)}")
  print(f"\nFormato final do DataFrame dados_OHE: {dados_OHE.shape}")
else:
  print(f"\nColunas com objetos categóricos precisam ser tratados: {', '.join(colunas_categoricas)}")

## Funções

### adiciona_janelas (<font color = 'yellow'>dados_anterior</font>, <font color = 'yellow'>dados_janela</font>, <font color = 'yellow'>colunas</font>=<font color = 'blue'>features_continuas_colunas</font>)
Mistura a base de duas janelas, permitindo fazer a avaliação de mais de uma janela para melhorar as previsões conforme o tempo de permanencia do visitante for aumentando.

In [ ]:
def adiciona_janela(dados_anterior, dados_janela, colunas=features_continuas_colunas):
  """
  ________________________________________________________________________________________________________________
  ENTRADAS
  --------
  dados_anterior: DataFrame
      dados da janela atual que está sendo observada. Por exemplo o filtro WINDOW == '0-2'

  dados_janela: DataFrame
      dados da janela que será adcionada. Seguindo o exemplo o filtro WINDOW == '2-4'

  colunas: list
      lista de colunas que devem ser adiciondos, não deveríamos adicionar novamente os dados categóricos por 
      exemplo. Por isso por padrão são utilizadas as features continuas
  ________________________________________________________________________________________________________________
  SAIDAS
  ------
  DataFrame
      Nova base de dados adcionando as colunas seleciondas, adicionando a variação delas contra a janela anterior 
      e removendo o que não for útil
  """
  d1 = dados_janela.set_index('PATIENT_VISIT_IDENTIFIER')
  d2 = dados_anterior.set_index('PATIENT_VISIT_IDENTIFIER')
  sufixo = ' ' + str(dados_anterior.loc[0,"WINDOW"])
  dados_novo = d1.join(d2[colunas], how='left', rsuffix=sufixo).reset_index()

  for _ in colunas:
    coluna = 'var ' + _ + sufixo
    coluna_ = _ + sufixo
    dados_novo[coluna] = dados_novo[_] - dados_novo[coluna_]

  descricao = dados_novo.describe().T
  colunas_sem_variacao = descricao[descricao['min'] == descricao['max']].index
  if len(colunas_sem_variacao) !=0:
    dados_novo.drop(colunas_sem_variacao, axis=1)
    print("\nRemovemos as colunas que os valores são iguais para todas as linhas")

  linhas_com_nam = dados_novo.describe(include='all').loc['count'].max()-dados_novo.describe(include='all').loc['count'].min()
  if linhas_com_nam !=0:
    if linhas_com_nam <= len(dados_novo)*.1:
      dados_novo.dropna(inplace=True)
      print(f"\nAs linhas ainda com Nam ({linhas_com_nam} linhas, {linhas_com_nam/len(dados_novo):.2%} do total) foram eliminadas")
    else:
      print(f"\nTemos linhas ainda com Nam ({linhas_com_nam} linhas, {linhas_com_nam/len(dados_novo):.2%} do total) precisam ser tratadas")
  print(f'\nBase nova de tamanho: {dados_novo.shape}')
  return dados_novo